# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Jan 20th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [4]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd

# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'
pwd
/data/pt_02586/sub_01/preprocessing


: 1

In [5]:
BVEC=$SUBJECTDIR'preprocessing/bvecs.txt';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';

BVEC=$SUBJECTDIR'preprocessing/bvecs.txt';
BVAL=$SUBJECTDIR'preprocessing/bvals.txt';


: 1

### DTI fit for QC

Computing a quick DTI fit and FA map is useful for QC.  
The DTI glyph are used to identify mismatch between the bvecs orientation and the data orientation  
The FA can be used to detect large anomaly.

#### Method 1: Look at tensor glyph for inconsistency

In [ ]:
# DTI/FA on non-eddy corrected data
dwi2tensor $SUBJECTDIR'preprocessing/dwi_denoise_degibbs.nii.gz' \
    $SUBJECTDIR'preprocessing/dti_quick.nii.gz' \
    -fslgrad $BVEC $BVAL

tensor2metric $SUBJECTDIR'preprocessing/dti_quick.nii.gz' \
    -fa $SUBJECTDIR'preprocessing/fa_quick.nii.gz'

#### Method 2: Use dwigradcheck
The dwigradcheck tool from MRtrix tries all possible permutation of the bvec and computes a few streamlines for each.  
Which ever orientation / flip produces the longuest streamline on average is chosen.  

Here I have manually produced various types of bad bvec to showcase the tool ability to detect them.

In [ ]:
# no flip
dwigradcheck dwi_denoise_degibbs.nii.gz \
    -mask mask_for_eddy.nii.gz \
    -fslgrad bvecs bvals \
    -export_grad_fsl bvecs_noflip_corrected bvals_noflip_corrected \
    -number 1000

In [ ]:
# x-flip (wrong sign on x axis of the bvec)
dwigradcheck dwi_denoise_degibbs.nii.gz \
    -mask mask_for_eddy.nii.gz \
    -fslgrad bvecs_xflip bvals \
    -export_grad_fsl bvecs_xflip_corrected bvals_xflip_corrected \
    -number 1000

In [ ]:
# y-flip (wrong sign on x axis of the bvec)
dwigradcheck dwi_denoise_degibbs.nii.gz \
    -mask mask_for_eddy.nii.gz \
    -fslgrad bvecs_yflip bvals \
    -export_grad_fsl bvecs_yflip_corrected bvals_yflip_corrected \
    -number 1000

In [ ]:
# z-flip (wrong sign on x axis of the bvec)
dwigradcheck dwi_denoise_degibbs.nii.gz \
    -mask mask_for_eddy.nii.gz \
    -fslgrad bvecs_zflip bvals \
    -export_grad_fsl bvecs_zflip_corrected bvals_zflip_corrected \
    -number 1000

### Susceptibility-induced off-resonance distorsion correction

#### What are we correcting?
Image distorsion at the brain boundaries.  
#### Why does this happen?
Diffusion imaging is typically performed using diffusion weighted spin-echo EPI (echo-planar imaging) images. These images will be very sensitive to non-zero off-resonance fields. Such fields will be caused by the susceptibility distribution of the subjects head (known as a susceptibility-induced off-resonance field).    
#### What if we ignore it?
In the typical case, where the phase encoding direction is AP, we will have large distorsion mainly at the front of the brain.  
#### Why correct it now?
This step can be performed at any moment before the EDDY step since it uses its own image for the estimation (at least one pair of AP and PA aquired b0).  
#### correction tool: topup
topup is the FSL tool to estimate the suceptibility induced distorsion.  
It takes in at least one pair of specially acquired b0 images with opposing phase encoding direction, for example AP and PA. It also required the phase encoding direction table that we extracted from the DICOMs at the very begining (we used the flag '-export_pe_table' with mrconvert and saved pe_table.txt)  
This pair of image as opposite types of suceptibility-induced distorsion and the tool computes a local deformation map that is consistent with both image and bring them on top of each other.  
Topup has a lots of parameters that are way too often ignored. The parameters are typically "hidden" inside a configuration file (more later).  
#### Quality control
- The AP and PA image after topup should overlap, if the distorsion looks worse, there was probably a sign wrong in the pe_table or the AP image was identified as the PA one and vice-versa (rare type of error). 


In [ ]:
# It is common to have the AP and PA aquisition as two separate files,
# so we need to concatenate the pe_table and the images
mrcat -axis 3 b0_AP.nii.gz b0_PA.nii.gz b0_AP_PA.nii.gz
cat pe_table_AP.txt pe_table_PA.txt > pe_table_AP_PA.txt

In [ ]:
# the typical topup call
# --imain: the input AP Pa image
# --datain: the pe_table
# --out: a prefix name for all the topup output files
# --iout: a name for the topup corrected b0_AP_PA image
# --config: a file with the rest of the parameters
topup --imain=b0_AP_PA.nii.gz \
      --datain=pe_table_AP_PA.txt \
      --out=topup_ \
      --iout=b0_topup.nii.gz \
      --config=b02b0_mod.cnf

In [1]:
cat /data/pt_02586/processing_tutorial/b02b0_mod.cnf

cat /data/pt_02586/processing_tutorial/b02b0_mod.cnf
# Resolution (knot-spacing) of warps in mm
--warpres=24.0,20.0,16.0,12.0,8.0,6.0,4.0,2.0
# Subsampling level (a value of 2 indicates that a 2x2x2 neighbourhood is collapsed to 1 voxel)
--subsamp=2,2,2,2,1,1,1,1
# FWHM of gaussian smoothing
--fwhm=4,3,3,2,1,0,0,0
# Maximum number of iterations
--miter=5,5,5,5,10,10,10,10
# Relative weight of regularisation
--lambda=0.001,0.0001,0.000015,0.000005,0.0000005,0.00000005,0.00000005,0.000000005
# If set to 1 lambda is multiplied by the current average squared difference
--ssqlambda=1
# Regularisation model
--regmod=bending_energy
# If set to 1 movements are estimated along with the field
--estmov=1,1,1,1,0,0,0,0
# 0=Levenberg-Marquardt, 1=Scaled Conjugate Gradient
--minmet=0,0,0,0,1,1,1,1
# Quadratic or cubic splines
--splineorder=3
# Precision for calculation and storage of Hessian
--numprec=double
# Linear or spline interpolation
--interp=spline
# If set to 1 the images are individually s

: 1

First note on the config files, topup is algorithm that make use of "multi-resolution", i.e. it does something at a coarse resolution because its fast, then it starts from the coarse solution and solves it again at better resolution using the fact that its faster to solve the high resolution case with the "hot start" solution from the coarse case.  
It keeps doing this more and more until we reach the resolution of the data.  
This is why all the parameters are 8 comma separated values in this case, they are the values for each resolution of the multi-resolution.  
Typically, you don't need/want to change this file much, the only really important aspects are:  
- same numbers of "step" for all the parameters  
- you want at least the last steps of "--subsamp" to be 1
- you want at least the last step of '--fwhm' to be 0  
- you want the last step of '--warpres' to be the data's resolution
- you want ALL the numbers in '--warpres' to be multiples of the data resolution (if data is 1.7mm, you want something like 20.4, 17.0, 13.6, 10.2, 6.8, 5.1, 3.4, 1.7 )

more info at https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/topup/TopupUsersGuide

After the topup estimation from the b0s, you can use the "applytopup" command to apply the same correction to all your dwi images, since they all have approximatively the same distorsions.  
However, we can do better. The next tool that we will use (eddy) has to option to take the topup field estimation as a parameter, and use it within it's own computation and output the fully corrected data.  
This is better since we don't need to run applytopup ourself and it reduces the total number of interpolation applied to our data (because the topup and eddy step are combined).

### Eddy current distorsion corrections

#### What are we correcting?
Image distorsion on each DWI volumes.    
#### Why does this happen?
The diffusion gradient (the physical gradient system on the machine, not the bvecs) create "accidental" extra magnetic field call eddy currents (https://en.wikipedia.org/wiki/Eddy_current) which translate into spatial image distorsion that depend on diffusion direction.     
#### What if we ignore it?
Mismatch between the different diffusion directions, especially visible at the brain edges and cotex.  
#### Why correct it now?
This step will require interpolation of the data and therefore need to be applied after the raw image correction such as denoising and degibbsing. The tool also requires the previously computed brain mask. Also, this tool can incorporate the previous topup correction.   
#### correction tool: eddy
eddy is the FSL tool to estimate and correct the eddy current induced distorsion. It also estimate and correct for subject motion.  
Eddy uses the fact that dwi with approximatively inverse bvec directions (like (x y z) and (-x -y -z) ) have approximatively the same diffusion contrast (diffusion is antipodally symmetric) but different eddy-current induced distorsions.  
It works optimally when the set of bvec direction is design to span a full sphere instead of just half a sphere (which is sadly common, because of the antipodal symmetry).  
Similar to topup, its uses two parameters textfile that we obtained from the initial dicom conversion (the eddy indices and the eddy index).  
The parameters are fairly self explanatory appart from:  
- "--data_is_shelled" flag which tells eddy that our data lives a shell of the same bvalue. This flag is neccesary because sometimes the bvals are slightly different between the dwi because of rounding and this will confuse eddy if we dont force it to consider it as a single bval shell.  
- "--dont_peas" flag which tell eddy not to run the "Post Eddy Alignment of Shells" which is a type of registration between the shells when you have multiple bvalues which I don't find very useful or stable.

#### Quality control
- When scrolling through the diffusion images, they should "move" less then before, both because of the motion correction and the eddy current distorsion correction.

In [ ]:
# Eddy is a VERY computationally expensive step
# The regular version of eddy is very slow
# However, there exist a version called eddy_openmp
# eddy_openmp is a multithreaded eddy version (parallel)
# there is also a cuda version (eddy_cuda8.0 in this case)
# which is a gpu implementation and its very fast if you have the hardware
# for example, on the ramones cluster at the mpi
# using the "CUDA --version 8.0" environement
# running eddy only takes 1-2 hours on a typical dataset
# sadly, you will have to figure out with your IT dept how to install this
# on whatever compute environement you use
# see "The eddy executables" section on
# https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy/UsersGuide
p_EDDY=/data/pt_02101_dMRI/software/eddy/eddy_cuda8.0

In [ ]:
# The first step is to take the optimal brain mask we previously computed
# In this case, the optimal threshold was store in a textfile betthforeddy.txt
# so I "build" the name of the file using that textfile
# after that, I apply 1 dilatation to this mask to make it a bit more permissive
maskfilter $SUBJECTDIR'preprocessing/b0_avg_N4x3_bet_init_0p'$( cat $SUBJECTDIR'preprocessing/betthforeddy.txt' )'_mask.nii.gz' \
    dilate \
    $SUBJECTDIR'preprocessing/mask_for_eddy.nii.gz' \
    -npass 1


In [ ]:
# eddy command
# it requires the data, the mask, the bval and bvec
# the special parameter textfile eddy indices and eddy config
# also, a name for all the output "-out"
# optionally, IF we had previously computed topup,
# there is a '--topup' parameter to give it the name used for the output
# for example "topup_"
${p_EDDY} --imain=$SUBJECTDIR'preprocessing/dwi_denoise_degibbs.nii.gz' \
          --mask=$SUBJECTDIR'preprocessing/mask_for_eddy.nii.gz' \
          --index=$SUBJECTDIR'preprocessing/eddy_ind.txt' \
          --acqp=$SUBJECTDIR'preprocessing/eddy_config.txt' \
          --bvecs=$BVEC \
          --bvals=$BVAL \
          --out=$SUBJECTDIR'preprocessing/dwi_eddy' \
          --interp=trilinear \
          --data_is_shelled \
          --dont_peas;

In [6]:
# Let's have a look at the multiple output of eddy
# It outputs various reports of motion correction, outlier detections, etc
# the only important files are 
# dwi_eddy.nii.gz: the corrected data
# dwi_eddy.eddy_rotated_bvecs: new bvecs taking the motion correction into account
# those will be the bvecs we use for everything starting now.
ll dwi_eddy.*

ll dwi_eddy.*
-rwxrwx--- 1 paquette users      2031 Jan 10 17:35 "dwi_eddy.eddy_movement_rms"*
-rwxrwx--- 1 paquette users     12634 Jan 10 17:34 "dwi_eddy.eddy_outlier_map"*
-rwxrwx--- 1 paquette users     52719 Jan 10 17:34 "dwi_eddy.eddy_outlier_n_sqr_stdev_map"*
-rwxrwx--- 1 paquette users     52252 Jan 10 17:34 "dwi_eddy.eddy_outlier_n_stdev_map"*
-rwxrwx--- 1 paquette users      2955 Jan 10 17:34 "dwi_eddy.eddy_outlier_report"*
-rwxrwx--- 1 paquette users     16698 Jan 10 17:34 "dwi_eddy.eddy_parameters"*
-rwxrwx--- 1 paquette users       637 Jan 10 17:34 "dwi_eddy.eddy_post_eddy_shell_alignment_parameters"*
-rwxrwx--- 1 paquette users       480 Jan 10 17:33 "dwi_eddy.eddy_post_eddy_shell_PE_translation_parameters"*
-rwxrwx--- 1 paquette users      2108 Jan 10 17:35 "dwi_eddy.eddy_restricted_movement_rms"*
-rwxrwx--- 1 paquette users      2813 Jan 10 17:34 "dwi_eddy.eddy_rotated_bvecs"*
-rwxrwx--- 1 paquette users 355669305 Jan 10 17:34 "dwi_eddy.nii.gz"*


: 1